### **Эксперимент 9.**

Построить отдельные модели для высот (ALT) 0-500, 500-1000, 1000-1500. Результат смотрим суммарно. 

Использовать конфигурацию эксперимента 5 (или 4, если лучше)


In [ ]:
import pandas as pd
import numpy as np 

y_cols = ['egtm']
meta_cols = ['reportts', 'acnum', 'pos']
features = ['egt', 'tat', 'n1a', 'n2a', 'wai', 'nai', 'prv', 'alt', 'mn', 'ff', 'nf']